In [1]:
import pandas as pd
import numpy as np
import os
import requests
import shutil
from tqdm import tqdm
from facebook_scraper import get_posts
import pythainlp as pythai

c:\GuideCH\Programming\project\aib_showcase\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('showcase_data_cleaned(2022).csv')

In [3]:
data.columns

Index(['title', 'ID', 'summary', 'title_image', 'images', 'links', 'reason',
       'author', 'fulltext', 'post_url', 'date'],
      dtype='object')

In [4]:
data.isna().sum()

title          0
ID             0
summary        0
title_image    0
images         0
links          0
reason         0
author         0
fulltext       0
post_url       0
date           0
dtype: int64

In [5]:
info_dict = {'date': "",  "title": "", "builder": "", "builder_info": "", "thumbnail": "", "links": {"github": "", "facebook": "", "blog": ""}, "title_image": "", "summary": "", "reason": ""}

In [6]:
for i in range(data.shape[0]):
    print(data['post_url'][i])
    break

https://facebook.com/aibuildersx/posts/405337768301335


In [7]:
def process_summary(x: str) -> list[str]:
    newx = x.split('- ')
    x = [x for x in newx if x != '']
    return newx

In [8]:
def process_link(link: str) -> list[str]:
    newx = link.split('\n')
    newx = [x for x in newx if x != '']
    medium = [x for x in newx if 'medium' in x or 'อ่านรายละเอียดเพิ่มเติม:' in x][0].split(': ')[-1]
    github = [x for x in newx if 'github' in x][0].split(': ')[-1]
    return medium, github

In [9]:
def process_author(link: str) -> list[str]:
    newx = link.split('/')
    name = newx[0]
    bio = '/'.join(newx[1:])
    return name, bio

In [10]:
posts = []
for project_index in tqdm(range(data.shape[0])):
    root = './images/2022/'

    info_dict = {'date': "",  "title": "", "builder": "", "builder_info": "", "thumbnail": "", "links": {"github": "", "facebook": "", "blog": ""}, "summary": "", "reason": ""}
    info_dict['date'] = data.iloc[project_index]['date']
    info_dict['title'] = data.iloc[project_index]['title']
    info_dict['builder'], data.iloc[project_index]['builder_info'] = process_author(data.iloc[project_index]['author'])
    # info_dict['thumbnail'] = data.iloc[project_index]['title_image']
    project_id = data.iloc[project_index]['ID'].astype(str)
    info_dict['thumbnail'] = root + project_id + '/' + '01' + '.jpg' # save to static/images/2022/21/01.jpg
    
    info_dict['links']['blog'], info_dict['links']['github'] = process_link(data.iloc[project_index]['links'])
    info_dict['links']['facebook'] = data.iloc[project_index]['post_url']
    info_dict['summary'] = process_summary(data.iloc[project_index]['summary'])
    info_dict['summary'] = new_summary = ['- ' + x.replace("'", '') for x in info_dict['summary'] if x != '']
    info_dict['reason'] = data.iloc[project_index]['reason'].split('แล้ว):')[-1].replace("',", '').replace("'", '')
    # strip everyshit
    for key in info_dict.keys():
        if type(info_dict[key]) == str:
            info_dict[key] = info_dict[key].strip()
        elif type(info_dict[key]) == list:
            info_dict[key] = [x.strip() for x in info_dict[key]]
        elif type(info_dict[key]) == dict:
            for key2 in info_dict[key].keys():
                info_dict[key][key2] = info_dict[key][key2].strip()
    
    posts.append(info_dict)

    # download image and save to static/images in format "/images/2022/21/01.jpg"
    # make folder according to ['ID']
    if os.path.exists(root) == False:
        os.mkdir(root)

    if os.path.exists(root + data.iloc[project_index]['ID'].astype(str)) == False:
        os.mkdir(root + data.iloc[project_index]['ID'].astype(str))

  0%|          | 0/45 [00:00<?, ?it/s]C:\Users\hirun\AppData\Local\Temp\ipykernel_30940\69931062.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_dict['builder'], data.iloc[project_index]['builder_info'] = process_author(data.iloc[project_index]['author'])
C:\Users\hirun\AppData\Local\Temp\ipykernel_30940\69931062.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_dict['builder'], data.iloc[project_index]['builder_info'] = process_author(data.iloc[project_index]['author'])
C:\Users\hirun\AppData\Local\Temp\ipykernel_30940\69931062.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [11]:
def make_md(data_dict):
    md_format ='''---
date: "{}"
title: "{}"
builder: "{}"
builder_info: "{}"
thumbnail: "{}"
links:
github: "{}"
facebook: "{}"
blog: "{}"
---

![image]({})

{}

### แรงจูงในในการเข้าร่วมโครงการ (จากใบสมัครเข้าร่วมเมื่อ 10 สัปดาห์ที่แล้ว)

> "{}"'''

    return md_format.format(data_dict['date'], data_dict['title'], data_dict['builder'], data_dict['builder_info'], data_dict['thumbnail'][1:], data_dict['links']['github'], data_dict['links']['facebook'], data_dict['links']['blog'], data_dict['thumbnail'][1:], '\n'.join(data_dict['summary']), info_dict['reason'])

In [12]:
data[data['title'] == 'MyLittleHR']

,title,ID,summary,title_image,images,links,reason,author,fulltext,post_url,date


In [13]:
for x in posts:
    print(make_md(x))

---
date: "1-7-22"
title: "BACK (Blind All Can Know) - Action Captioning for Blinds"
builder: "โชติวัฒน์ ตั้งสถาพร (ไนน์)"
builder_info: ""
thumbnail: "/images/2022/21/01.jpg"
links:
github: "https://colab.research.google.com/github/cninet/BACK/blob/main/BACK.ipynb"
facebook: "https://facebook.com/aibuildersx/posts/405337768301335"
blog: "https://medium.com/@cninet.std/back-blind-all-can-know-action-captioning-a10a3fa85695"
---

![image](/images/2022/21/01.jpg)

- โมเดลอธิบายรูปภาพ (image captioning) เพื่ออธิบายภาพตรงหน้าให้กับผู้พิการทางสายตา,
- CLIPCap นำ pretrained CLIP เพื่อสร้าง image embeddings เป็น input ให้กับ pretrained GPT2 สร้างข้อความที่ตรงกับภาพ,
- เทรนโมเดลด้วย Flickr30k (รูป-คำบรรยายภาษาอังกฤษ) เป็นเวลา 17 ชั่วโมง (10 epochs),
- ตัดสินใจใช้ Flickr30k ทั้งที่จำนวนข้อมูลน้อยกว่า COCO เพราะทำแบบสอบถามแล้วพบว่าข้อมูลคำบรรยายของ Flickr30k มีคุณภาพมากกว่า,
- ใช้ PyThaiNLP Translate ในการแปลภาษาคำบรรยายเป็นภาษาไทย เนื่องจากแบบสอบถามพบว่าเป็นธรรมชาติกว่า Google Translate API ในบ

In [14]:
def naming_sense(text, root='./posts/2022/'):
    # tokenize
    tokens = pythai.word_tokenize(text)
    x = ''.join(tokens[:6])
    if len(x) < 30:
        x = text
    elif len(x) > 30:
        x = x = ''.join(tokens[:3])
        
    naming = x.replace(' ', '').replace('!', '').replace('?', '').replace(':','')
    path = f"{root}{naming}.md"
    return path

for post in posts:
    data = make_md(post)
    if naming_sense(post['title']) in ['ChaosEDM', 'Crossec', 'Debunker', 'Edge-to-Face', 'MyLittleHR', 'NLPforgenrepredictionsonFFnet', 'PsychNLP', 'TextDoe', 'vTranslator']:
        print(post['title'])

    # test naming
    print(naming_sense(post['title']))
    with open(naming_sense(post['title'], "./posts/2022/"), 'w', encoding='utf8') as f:
        f.write(data)

./posts/2022/BACK(BlindAllCanKnow)-ActionCaptioningforBlinds.md
./posts/2022/ViolenceDetection.md
./posts/2022/FakeProductDetectiononOnlineRetail.md
./posts/2022/CactusClassificationwithFastai.md
./posts/2022/FoodIngredientsLabelReaderforFoodAllergy.md
./posts/2022/Edge-to-Face.md
./posts/2022/TextDoeโมเดลจำแนกแวดวงบทความสิ่งพิมพ์.md
./posts/2022/AutoLyricRecognizer.md
./posts/2022/ImageTransferDay-to-NightandNight-to-Day.md
./posts/2022/MusicRecognition.md
./posts/2022/vTranslatorTranscribeandTranslateVTuberusingWav2Vec2.md
./posts/2022/Text-to-imagesynthesiswithVQGAN-ThCLIP.md
./posts/2022/GameRecommendationbyusingNeuralNetworkEmbeddings.md
./posts/2022/ChaosEDMGeneratingEDMSongwithVAE(VariationalAutoencoder)Spectrogram.md
./posts/2022/WanchanBERTaThaiGrammarly.md
./posts/2022/GarbageDetectionwithTensorflowLite.md
./posts/2022/SickPigClassifier.md
./posts/2022/NLPforgenrepredictionsonFFnetanantithesistoutilitarianism.md
./posts/2022/AIแยกแยะแมงดาจานกับแมงดาถ้วย.md
./posts/2022/Isthat

In [15]:
posts[0]['summary']

['- โมเดลอธิบายรูปภาพ (image captioning) เพื่ออธิบายภาพตรงหน้าให้กับผู้พิการทางสายตา,',
 '- CLIPCap นำ pretrained CLIP เพื่อสร้าง image embeddings เป็น input ให้กับ pretrained GPT2 สร้างข้อความที่ตรงกับภาพ,',
 '- เทรนโมเดลด้วย Flickr30k (รูป-คำบรรยายภาษาอังกฤษ) เป็นเวลา 17 ชั่วโมง (10 epochs),',
 '- ตัดสินใจใช้ Flickr30k ทั้งที่จำนวนข้อมูลน้อยกว่า COCO เพราะทำแบบสอบถามแล้วพบว่าข้อมูลคำบรรยายของ Flickr30k มีคุณภาพมากกว่า,',
 '- ใช้ PyThaiNLP Translate ในการแปลภาษาคำบรรยายเป็นภาษาไทย เนื่องจากแบบสอบถามพบว่าเป็นธรรมชาติกว่า Google Translate API ในบริบทนี้,',
 '- ใช้ Google TTS ในการเปลี่ยนคำบรรยายที่ถูกแปลเป็นเสียงพูด,',
 '- นอกจากการเทรนโมเดล CLIPCap ตรง ๆ แล้วยังมีการนำมาประกอบการใช้งานกับ pretrained models อื่น ๆ เช่น PyThaiNLP Translate และ Google TTS อีกทั้งการต่อสู้กับคุณภาพข้อมูลอย่างสมศักดิ์ศรี เช่น ลบรูปซ้ำ, แก้ข้อมูลตาราง, แก้คำบรรยายที่มีมากกว่าหนึ่งประโยคต่อกัน ฯลฯ']